In [1]:
#!conda install -n tf_2.3.1 python=3.8.3
#!conda activate tf_2.3.1
#!pip install --upgrade tensorflow-2.3.1
#!conda install scikit-learn
#!pip install ~/cloudfiles/code/software/PyPeanuts/latest/peanuts-0.6.5-py3-none-any.whl --force-reinstall

In [2]:
import pandas as pd
import peanuts
from peanuts.AML.orion import *

import tensorflow as tf
print(tf.version.VERSION) # shuold be 2.3.1
from model import EfficientGAN

username='746220'
orion = Orion(user=username)

#import umap
#import umap.plot
#import matplotlib.pyplot as plt

2022-04-07 22:32:38.764492: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-04-07 22:32:38.764560: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.3.1
🔔 Just a Reminder: Is your key vault name following the naming convention: ba-<n|p>-z<eaus|weus>-<user>-kv? If NO, please specify your key vault name.
🔔 Just a Reminder: Do you have a secret with name corpaaid-pw stored in your key vault? If NO, please specify your secret name.


In [3]:
def sql_path_loader(data_source, scoring_training_ind):
    """
    This function loads query paths based on input args.
    We have saved queries in these locations.
    """
    
    if data_source == 'trans':
      if scoring_training_ind=='training':
        sql_path="../AWD_TRANS_Databricks_Query.sql"  # _Feb24-1.txt"
      else:
        # sql_path="/dbfs/FileStore/tables/data_query_test_Mar4.txt"
        sql_path="../AWD_TRANS_Databricks_Query.sql"
#       data_query_updated_Feb-1.txt"
      #data_query_Jan-3.txt" data_query_Jan_vol.txt"  data_query_Jan_vol_cast.txt
#     elif data_source == "clickstream":
#             sql_path = "./"
    return sql_path

# sql_path_loader('trans')

def sql_renderer(data_source, scoring_training_ind, start_date, end_date, print_sql=False):
    """
    This function renders the query based on input args
    """
    
    sql_path = sql_path_loader(data_source = data_source, scoring_training_ind=scoring_training_ind )
    sql = open(sql_path, 'r').read()
#     if scoring_training_ind=='training':
    sql = sql.format(lb=start_date, ub=end_date)
#     else:
#       sql = sql.format(CURRENT_DATE=date)
    if print_sql:
        print(sql)
    return sql
  
def data_retriever_fun(data_source,scoring_training_ind, start_date, end_date, print_sql):
    """
    This function retrieves records from mosaic tables
    """
        
    if (data_source != "trans") & (data_source != "clickstream"):
        print("### ERROR: data source should be either trans or clickstream ###")
        return
    elif (start_date == None):
        print("### ERROR: Provide date in format of 'yyyy-mm-dd' ###")
        return
    else:
            sql = sql_renderer(data_source = data_source, scoring_training_ind=scoring_training_ind, 
            start_date=start_date, end_date=end_date, print_sql = print_sql)
            retrieved_df = orion.mq(sql)#pd.read_sql(sql)
            return retrieved_df

In [4]:
sdf_test=data_retriever_fun(data_source='trans', scoring_training_ind='scoring', start_date='2022-03-30', end_date='2022-04-05', print_sql=False)
sdf=data_retriever_fun(data_source='trans', scoring_training_ind='training', start_date='2022-02-28', end_date='2022-03-29', print_sql=False)

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy


In [5]:
# sdf.display()
# without filter
indices=['LYLTY_ACCT_ID', 'PNR_LOCTR_ID', 'PNR_CREATE_DT', 'AWD_PKG_ISSUE_DT']
categorical=['OPERAT_AIRLN','ORIGIN_RM_WRLD_REGION_CD','DESTNTN_RM_WRLD_REGION_CD']
#, 'OPERAT_AIRLN_IATA_CD2', 'OD_ORIGIN_AIRPRT_IATA_CD','OD_DESTNTN_AIRPRT_IATA_CD','ORIGIN_CNTRY_CD','ORIGIN_RM_WRLD_REGION_CD','DESTNTN_CNTRY_CD','DESTNTN_RM_WRLD_REGION_CD'
#  'VISITED_BEFORE_STATE'
#'AWD_LEVEL_MILE_QTY','NUM_PAX','SAFE_AIRPRT_IND','FLAGGED_AIRPRT_IND', 
numericals=['AP','PAX_LYLTY_IND', 
            'VISITED_BEFORE_AIRPORT',
            #'VISITED_BEFORE_CNTRY',
            'USED_BEFORE_ARILN',
            'ONE_WAY', 'SHORT_AP_OK',
            'SEEN_ITIN',
            #'SAFE_VS_FLAGGED_CNTRY', 
            'BIN_HIST', 'MULTI_REDEEM', 'EMAIL_CHANGE30', 
            'U_AP', 'AGE45', 'AGE_logEMAIL',
            'LAST_EMAIL',
            'AGE_LAST_EMAIL',
            #'HOME_AIRPRT',
            #'HOME_STATE',
            #'HOME_RESDNC_CNTRY',
            #'HOME_ADV',
            'FLAGGED_CNTRY_ALERT',
            'FLAGGED_AIRPRT_ALERT',
            'risk_feature1', 
            'risk_feature2',
            'risk_feature3',
            'risk_feature4',
            #'risk_feature5',
            'risk_feature6',
 #           'safe_feature1',
 #           'FREQ_USED_ARILN',
 #           'FREQ_DESTNTN_CNTRY',
 #           'FREQ_ORIGIN_CNTRY',
 #           'FREQ_CNTRY',
#            'FREQ_ORIGIN_AIRPRT',
#            'FREQ_DESTNTN_AIRPRT',
 #           'FREQ_AIRPORT',
           ]

# 'FLAGGED_CNTRY_ALERT', 'SAFE_CNTRY_OK',
#  'SAFE_VS_FLAGGED_CNTRY',
#  'NUM_HIST',
#  'DAYS_SINCE_LAST'
predictors=numericals+categorical

In [6]:
#sdfNorm = sdf.query('(BIN_HIST == 1) & (NORMAL_TRAN == 1)')
sdfNorm = sdf.query('(BIN_HIST == 1) & ((NORMAL_TRAN == 1) | (FLOWN_LAST_NM == 1) | (RED_LAST_NM == 1) | (IP_MATCH == 1))')

sdfNorm = sdfNorm.drop_duplicates()
#sdfFiltered = sdf.filter((sdf.OWNER_IN_FLIGHT != 1) & (sdf.BIN_HIST == 1))

#sdf_test_clean = sdf_test.query('((BIN_HIST == 1) & (RAW_AP <= 9) & ~(NORMAL_TRAN == 1))')
sdf_test_clean = sdf_test.query('((BIN_HIST == 1) & (RAW_AP <= 9) & ~((NORMAL_TRAN == 1) | (FLOWN_LAST_NM == 1) | (RED_LAST_NM == 1) | (IP_MATCH == 1)))')
sdf_test_clean = sdf_test_clean.drop_duplicates()
#predictions=model.transform(sdf_test_clean)

#& sdf.OWNER_IN_FLIGHT != 1 & sdf.PAX_REDEEMED_BEFORE != 1 & sdf.ONE_WAY != 1)

numAfter = len(sdfNorm)
numBefore = len(sdf)
print(f"rows of normal instances {numAfter}")
print(f"rows of all instances {numBefore}")
print(f"reduction percentage AFTER filtering {round(numAfter / numBefore * 100, 2)}")

print(f"rows AFTER filtering {len(sdf_test_clean)}")
print(f"rows BEFORE filtering {len(sdf_test)}")

rows of normal instances 346179
rows of all instances 556564
reduction percentage AFTER filtering 62.2
rows AFTER filtering 1125
rows BEFORE filtering 95253


In [7]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder# creating initial dataframe
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

df = sdfNorm
dfTest = sdf_test_clean

le = LabelEncoder()
dfCopy = df[predictors].copy()
dfTestCopy = dfTest[predictors].copy()

#for col in categorical:
#  dfCopy[col] = le.fit_transform(dfCopy[col])
#  dfTestCopy[col] = le.fit_transform(dfTestCopy[col])

ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
ohTrain = pd.DataFrame(ohe.fit_transform(dfCopy[categorical]))
ohTest = pd.DataFrame(ohe.transform(dfTestCopy[categorical]))

# Adding column names to the encoded data set.
ohTrain.columns = ohe.get_feature_names_out(categorical)
ohTest.columns = ohe.get_feature_names_out(categorical)

# One-hot encoding removed index; put it back
ohTrain.index = dfCopy.index
ohTest.index = dfTestCopy.index

# Remove categorical columns (will replace with one-hot encoding)
dfCopy = dfCopy.drop(categorical, axis=1)
dfTestCopy = dfTestCopy.drop(categorical, axis=1)

# Add one-hot encoded columns to numerical features
dfCopy = pd.concat([dfCopy, ohTrain], axis=1)
dfTestCopy = pd.concat([dfTestCopy, ohTest], axis=1)

In [8]:
dfTestCopy.shape
dfCopy.shape
dfCopy.columns

Index(['AP', 'PAX_LYLTY_IND', 'VISITED_BEFORE_AIRPORT', 'USED_BEFORE_ARILN',
       'ONE_WAY', 'SHORT_AP_OK', 'SEEN_ITIN', 'BIN_HIST', 'MULTI_REDEEM',
       'EMAIL_CHANGE30', 'U_AP', 'AGE45', 'AGE_logEMAIL', 'LAST_EMAIL',
       'AGE_LAST_EMAIL', 'FLAGGED_CNTRY_ALERT', 'FLAGGED_AIRPRT_ALERT',
       'risk_feature1', 'risk_feature2', 'risk_feature3', 'risk_feature4',
       'risk_feature6', 'OPERAT_AIRLN_0', 'OPERAT_AIRLN_1', 'OPERAT_AIRLN_2',
       'ORIGIN_RM_WRLD_REGION_CD_AF', 'ORIGIN_RM_WRLD_REGION_CD_AS',
       'ORIGIN_RM_WRLD_REGION_CD_CA', 'ORIGIN_RM_WRLD_REGION_CD_CR',
       'ORIGIN_RM_WRLD_REGION_CD_EU', 'ORIGIN_RM_WRLD_REGION_CD_ME',
       'ORIGIN_RM_WRLD_REGION_CD_NA', 'ORIGIN_RM_WRLD_REGION_CD_OC',
       'ORIGIN_RM_WRLD_REGION_CD_SA', 'DESTNTN_RM_WRLD_REGION_CD_AF',
       'DESTNTN_RM_WRLD_REGION_CD_AS', 'DESTNTN_RM_WRLD_REGION_CD_CA',
       'DESTNTN_RM_WRLD_REGION_CD_CR', 'DESTNTN_RM_WRLD_REGION_CD_EU',
       'DESTNTN_RM_WRLD_REGION_CD_ME', 'DESTNTN_RM_WRLD_REGION_C

In [9]:
model = EfficientGAN()
model.fit(dfCopy, epochs=500, batch_size=1024)
proba = model.predict(dfTestCopy)
dfTest['decision_function'] = proba

2022-04-07 22:49:45.634928: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-04-07 22:49:46.667680: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-07 22:49:46.667778: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ba-p-zeaus-746220sazmlci): /proc/driver/nvidia/version does not exist
2022-04-07 22:49:46.669090: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-07 22:49:46.762849: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2095245000 Hz
2022-04-07 22:49:

epoch0-> d_loss:0.9358741790056229  e_loss:13.949773788452148  g_loss:0.6118311882019043
epoch10-> d_loss:0.9526254236698151  e_loss:10.719099998474121  g_loss:0.6175879240036011
epoch20-> d_loss:1.156915545463562  e_loss:9.620051383972168  g_loss:0.6180082559585571
epoch30-> d_loss:1.3672295808792114  e_loss:7.5071563720703125  g_loss:0.6075266599655151
epoch40-> d_loss:1.7185212969779968  e_loss:6.337667465209961  g_loss:0.6169198751449585
epoch50-> d_loss:2.0528809428215027  e_loss:5.112499237060547  g_loss:0.6026747226715088
epoch60-> d_loss:2.3990713357925415  e_loss:4.306389331817627  g_loss:0.6154841780662537
epoch70-> d_loss:2.5784838795661926  e_loss:3.483208179473877  g_loss:0.6087245345115662
epoch80-> d_loss:2.8176766633987427  e_loss:3.380417585372925  g_loss:0.6036961078643799
epoch90-> d_loss:3.277631103992462  e_loss:3.2735955715179443  g_loss:0.6128025650978088
epoch100-> d_loss:3.5771483182907104  e_loss:2.967963218688965  g_loss:0.6060472726821899
epoch110-> d_loss:3

In [10]:
out = dfTest.copy()
#out = out.nsmallest(n=100, columns='decision_function').copy()
#out.groupby(out.LYLTY_ACCT_ID).first().display()
out = dfTest.groupby(out.LYLTY_ACCT_ID).apply(lambda x: x.loc[x['decision_function'].idxmax()]).nlargest(n=100, columns='decision_function').copy()
#out = out.reset_index()
out.to_csv('../GANres220330-040.csv')

In [11]:
dfTest.nlargest(n=100, columns='decision_function')

,LYLTY_ACCT_ID,PNR_LOCTR_ID,PNR_CREATE_DT,AWD_PKG_ISSUE_DT,AWD_LEVEL_MILE_QTY,NUM_PAX,OD_ORIGIN_AIRPRT_IATA_CD,OD_DESTNTN_AIRPRT_IATA_CD,AP,RAW_AP,...,LAST_EMAIL,AGE_LAST_EMAIL,risk_feature1,risk_feature2,risk_feature3,risk_feature4,risk_feature5,risk_feature6,safe_feature1,decision_function
4817,M546732,FKSZNB,2022-04-01,2022-04-01,25000,1,CVG,CLT,0,0,...,2.346353,77.346353,1.880814,3.333333,1.465980,154.692706,2.000000,158.795880,9,1260.282427
34387,PDB4246,HHEZUP,2022-03-26,2022-03-26,60000,1,CDG,LAX,7,7,...,3.328380,96.328380,-5.026872,1.000000,0.477815,192.656759,0.250000,24.444538,14,1204.476045
18976,965T5L4,UPMFKA,2022-04-01,2022-04-01,55000,2,SXM,MIA,2,2,...,0.000000,71.000000,-2.538867,2.000000,0.948073,284.000000,1.333333,56.143488,8,1082.116265
27643,965T5L4,UPMFKA,2022-04-01,2022-04-01,55000,2,SXM,MIA,2,2,...,0.000000,71.000000,-2.538867,2.000000,0.948073,284.000000,1.333333,56.143488,8,1082.116265
8167,965T5L4,VUEINX,2022-04-01,2022-04-01,30000,1,SXM,MIA,2,2,...,0.000000,71.000000,-2.538867,2.000000,0.895424,284.000000,1.333333,55.948357,8,1081.767368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68171,2E34VP2,SRRHGO,2022-03-24,2022-03-25,50000,1,SFO,CLT,8,8,...,-0.211229,83.788771,-9.310381,0.909091,0.427179,83.788771,0.111111,7.246766,10,593.827710
33139,2KB1E70,HSUKDP,2022-04-02,2022-04-02,63000,3,SRQ,STL,1,1,...,-0.000664,84.999336,-1.881743,2.500000,1.199835,84.999336,0.500000,18.645108,14,593.491489
12906,287T7R4,CBFBUR,2022-03-27,2022-03-28,27500,1,SXM,MIA,8,8,...,-1.074486,61.925514,-8.759668,0.476190,0.211397,123.851028,0.222222,8.665038,12,592.041467
56017,4AF6858,VSPLBB,2022-03-23,2022-03-23,121000,1,MIA,CMH,8,8,...,1.355643,81.355643,-7.091515,1.818182,0.462071,81.355643,0.111111,7.734799,9,587.903548


In [12]:
#import umap
#import umap.plot
#import matplotlib.pyplot as plt

#mapper = umap.UMAP(random_state=42).fit(dfTestCopy)
#umap.plot.points(mapper, labels=dfTestCopy['decision_function']>800)

ImportError: Numba needs NumPy 1.21 or less

In [ ]:
#clf = IsolationForest(n_estimators=100, max_samples=4096, random_state=0).fit(dfCopy[predictors])

#dd = pd.DataFrame(clf.decision_function(dfTestCopy[predictors]))
#dfTest['decision_function'] = dd
#out = dfTest.nsmallest(n=100, columns='decision_function')

In [ ]:
import sys
print(sys.version)